In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import concurrent.futures
from selenium.common.exceptions import StaleElementReferenceException


In [2]:
data = {
    "keywords": "anies",
    "since_time": "2023-01-01",
    "until_time": "2023-11-10"
}

In [3]:
def scrape_pagination(keywords, since_time, until_time):
    current_date = datetime.strptime(since_time, "%Y-%m-%d")
    until_date = datetime.strptime(until_time, "%Y-%m-%d")
    result_data = []  # Membuat list kosong untuk menyimpan data
    
    while current_date <= until_date:
        formatted_date = current_date.strftime("%d/%m/%Y")
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={formatted_date}&todatex={formatted_date}&page=1"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        cek = soup.find('span', {"class": "fl text"})
        
        if cek:
            hasil_text = cek.get_text()
            result = re.search(r'\d+', hasil_text)
            hasil_number = int(result.group(0))
            hasil_divided = hasil_number / 9
            page_index = round(hasil_divided)
            print(formatted_date)
            result_data.append({
                'keywords':keywords,
                'tanggal_berita': formatted_date,
                'jumlah_index': page_index
            })
        else:
            print(f"No data found for {formatted_date}")
        current_date += timedelta(days=1)
    
    return result_data 

In [4]:
data_tanggal = scrape_pagination(data["keywords"], data["since_time"], data["until_time"])
print(data_tanggal)

01/01/2023
02/01/2023
03/01/2023
04/01/2023
05/01/2023
06/01/2023
07/01/2023
08/01/2023
09/01/2023
10/01/2023
11/01/2023
12/01/2023
13/01/2023
14/01/2023
15/01/2023
16/01/2023
17/01/2023
18/01/2023
19/01/2023
20/01/2023
21/01/2023
22/01/2023
23/01/2023
24/01/2023
25/01/2023
26/01/2023
27/01/2023
28/01/2023
29/01/2023
30/01/2023
31/01/2023
01/02/2023
02/02/2023
03/02/2023
04/02/2023
05/02/2023
06/02/2023
07/02/2023
08/02/2023
09/02/2023
10/02/2023
11/02/2023
12/02/2023
13/02/2023
14/02/2023
15/02/2023
16/02/2023
17/02/2023
18/02/2023
19/02/2023
20/02/2023
21/02/2023
22/02/2023
23/02/2023
24/02/2023
25/02/2023
26/02/2023
27/02/2023
28/02/2023
01/03/2023
02/03/2023
03/03/2023
04/03/2023
05/03/2023
06/03/2023
07/03/2023
08/03/2023
09/03/2023
10/03/2023
11/03/2023
12/03/2023
13/03/2023
14/03/2023
15/03/2023
16/03/2023
17/03/2023
18/03/2023
19/03/2023
20/03/2023
21/03/2023
22/03/2023
23/03/2023
24/03/2023
25/03/2023
26/03/2023
27/03/2023
28/03/2023
29/03/2023
30/03/2023
31/03/2023
01/04/2023

In [5]:
def scrape_links(page_number, keywords, date, link_list):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={date}&todatex={date}&page={page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')

    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)

    print(f"Scraped {len(page_links)} links from page {page_number} cek tanggal {date}")
    
    # Extend the link_list with the links scraped on this page
    link_list.extend(page_links)

In [6]:
def scrape_all_links(data_tanggal):
    link_list = []  
    with ThreadPoolExecutor(max_workers=8) as executor:  
        for item in data_tanggal:
            keywords = item['keywords']
            date = item['tanggal_berita']
            max_pages = item['jumlah_index']
            for page_number in range(1, max_pages+1):  
                executor.submit(scrape_links, page_number, keywords, date, link_list)
    
    return link_list

In [7]:
all_links = scrape_all_links(data_tanggal)
print(len(all_links))

Scraped 8 links from page 1 cek tanggal 08/01/2023
Scraped 7 links from page 1 cek tanggal 06/01/2023
Scraped 9 links from page 1 cek tanggal 02/01/2023
Scraped 6 links from page 2 cek tanggal 04/01/2023
Scraped 6 links from page 2 cek tanggal 05/01/2023
Scraped 9 links from page 1 cek tanggal 04/01/2023
Scraped 9 links from page 1 cek tanggal 03/01/2023
Scraped 9 links from page 1 cek tanggal 05/01/2023
Scraped 9 links from page 1 cek tanggal 13/01/2023
Scraped 9 links from page 1 cek tanggal 11/01/2023
Scraped 5 links from page 2 cek tanggal 12/01/2023
Scraped 9 links from page 1 cek tanggal 10/01/2023
Scraped 5 links from page 1 cek tanggal 09/01/2023
Scraped 4 links from page 2 cek tanggal 11/01/2023
Scraped 9 links from page 1 cek tanggal 12/01/2023
Scraped 4 links from page 2 cek tanggal 16/01/2023
Scraped 9 links from page 1 cek tanggal 18/01/2023
Scraped 6 links from page 3 cek tanggal 17/01/2023
Scraped 9 links from page 1 cek tanggal 19/01/2023
Scraped 9 links from page 1 cek

In [8]:
def scrape_url(url, keywords,max_retries=3):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')

                    # Extract data from the web page
                    title_elem = soup.find('h1', {"class": "detail__title"})
                    title_text = title_elem.text.strip() if title_elem else "Title not found"

                    date_elem = soup.find('div', {"class": "detail__date"})
                    date_text = date_elem.text.strip() if date_elem else "Date not found"

                    body_elem = soup.find('div', {"class": "detail__body"})
                    if body_elem:
                        content_elem = body_elem.find_all('p')
                        content_text = "\n".join(p.text.strip() for p in content_elem)
                        content_text = content_text.replace('\n', ' ')
                    else:
                        content_text = "Content not found"

                    return {
                        'title': title_text,
                        'keywords': keywords,
                        'date': date_text,
                        'content': content_text,
                        'nama_berita': 'detik',
                        'link': url
                    }
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None         

In [9]:
# Create a list to store the results
results = []

# Define a function to scrape URLs and store results
def scrape_and_store_result(url):
    result = scrape_url(url, data['keywords'])
    if result:
        results.append(result)

# Create a ThreadPoolExecutor to run the scraping function concurrently
max_threads = 8  # You can adjust the number of threads as needed
with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
    executor.map(scrape_and_store_result, all_links)

# Print or process the results as needed
for result in results:
    print(result)

Failed to retrieve data from https://news.detik.com/detiktv/d-6836931/anies-petik-pelajaran-usai-nonton-timnas-voli-indonesia-vs-vietnam: Status Code 404
Retrying https://news.detik.com/detiktv/d-6836931/anies-petik-pelajaran-usai-nonton-timnas-voli-indonesia-vs-vietnam (Attempt 1/3)
Failed to retrieve data from https://news.detik.com/detiktv/d-6836931/anies-petik-pelajaran-usai-nonton-timnas-voli-indonesia-vs-vietnam: Status Code 404
Retrying https://news.detik.com/detiktv/d-6836931/anies-petik-pelajaran-usai-nonton-timnas-voli-indonesia-vs-vietnam (Attempt 2/3)
Failed to retrieve data from https://news.detik.com/detiktv/d-6836931/anies-petik-pelajaran-usai-nonton-timnas-voli-indonesia-vs-vietnam: Status Code 404


In [ ]:
results

[{'title': 'Sandiaga Ngaku Masih Kader Gerindra, Dasco: Prabowo Capres Tunggal',
  'keywords': 'prabowo',
  'date': '25,050 Views  |  Selasa, 03 Jan 2023 18:13 WIB',
  'content': 'Sandiaga Uno mengaku masih menjadi kader Gerindra usai ramai desas-desus bergabung dengan ke PPP. Ketua Harian Gerindra, Sufmi Dasco Ahmad menegaskan capres dari Partai Gerindra adalah Prabowo Subianto.',
  'nama_berita': 'detik',
  'link': 'https://news.detik.com/detiktv/d-6496904/sandiaga-ngaku-masih-kader-gerindra-dasco-prabowo-capres-tunggal'},
 {'title': 'Elektabilitas Ganjar, Anies dan Prabowo Setahun Terakhir Versi Indo Riset',
  'keywords': 'prabowo',
  'date': '20,204 Views  |  Selasa, 03 Jan 2023 16:15 WIB',
  'content': 'Lembaga Indo Riset merilis hasil survei elektabilitas calon presiden dalam satu tahun terakhir. Hasilnya Ganjar Pranowo menempati posisi pertama. ganjar disusul Anies Baswedan dan Prabowo Subianto.',
  'nama_berita': 'detik',
  'link': 'https://news.detik.com/detiktv/d-6496587/elek

In [ ]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df

hasil scrapping 13117


,title,keywords,date,content,nama_berita,link
0,"Sandiaga Ngaku Masih Kader Gerindra, Dasco: Pr...",prabowo,"25,050 Views | Selasa, 03 Jan 2023 18:13 WIB",Sandiaga Uno mengaku masih menjadi kader Gerin...,detik,https://news.detik.com/detiktv/d-6496904/sandi...
1,"Elektabilitas Ganjar, Anies dan Prabowo Setahu...",prabowo,"20,204 Views | Selasa, 03 Jan 2023 16:15 WIB",Lembaga Indo Riset merilis hasil survei elekta...,detik,https://news.detik.com/detiktv/d-6496587/elekt...
2,Prabowo Ungkap Urusan Kemhan dan TNI Saat Masa...,prabowo,"Selasa, 03 Jan 2023 21:04 WIB",Menhan Prabowo Subianto mengawali 2023 dengan ...,detik,https://news.detik.com/berita/d-6496993/prabow...
3,Terkuak Bocah Jakpus Diculik Hampir Sebulan Ke...,prabowo,"Selasa, 03 Jan 2023 21:22 WIB","Bocah korban penculikan di Gunung Sahari, Jaka...",detik,https://news.detik.com/berita/d-6497026/terkua...
4,Rahayu dan Politisi PDIP Kompak Soroti Survei ...,prabowo,"Selasa, 03 Jan 2023 16:47 WIB",Lembaga survei Indo Riset telah merilis hasil ...,detik,https://news.detik.com/pemilu/d-6496532/rahayu...
...,...,...,...,...,...,...
13112,TKN Prabowo-Gibran Pede Adu Kuat Suara NU deng...,prabowo,"89,344 Views | Jumat, 10 Nov 2023 04:58 WIB",Sekretaris Tim Kampanye Nasional (TKN) Prabowo...,detik,https://news.detik.com/detiktv/d-7028676/tkn-p...
13113,Jokowi Beri Kehormatan Bintang Jasa Pratama ke...,prabowo,"Jumat, 10 Nov 2023 10:27 WIB",Presiden Joko Widodo (Jokowi) memberikan penga...,detik,https://news.detik.com/berita/d-7028939/jokowi...
13114,"Usai Upacara Hari Pahlawan, Jokowi dan Ma'ruf ...",prabowo,"Jumat, 10 Nov 2023 09:10 WIB",Presiden Joko Widodo (Jokowi) selesai memimpin...,detik,https://news.detik.com/berita/d-7028834/usai-u...
13115,Ingin Anaknya Masuk Polri Bikin Janda di Medan...,prabowo,"Jumat, 10 Nov 2023 09:00 WIB",Keinginan Sergina Sitorus agar anaknya bisa ma...,detik,https://www.detik.com/sumut/berita/d-7028627/i...


In [ ]:
filtered_df = df[df['content'].str.contains(data["keywords"], case=False)|
                 df['title'].str.contains(data["keywords"], case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 12569


,title,keywords,date,content,nama_berita,link
0,"Sandiaga Ngaku Masih Kader Gerindra, Dasco: Pr...",prabowo,"25,050 Views | Selasa, 03 Jan 2023 18:13 WIB",Sandiaga Uno mengaku masih menjadi kader Gerin...,detik,https://news.detik.com/detiktv/d-6496904/sandi...
1,"Elektabilitas Ganjar, Anies dan Prabowo Setahu...",prabowo,"20,204 Views | Selasa, 03 Jan 2023 16:15 WIB",Lembaga Indo Riset merilis hasil survei elekta...,detik,https://news.detik.com/detiktv/d-6496587/elekt...
2,Prabowo Ungkap Urusan Kemhan dan TNI Saat Masa...,prabowo,"Selasa, 03 Jan 2023 21:04 WIB",Menhan Prabowo Subianto mengawali 2023 dengan ...,detik,https://news.detik.com/berita/d-6496993/prabow...
4,Rahayu dan Politisi PDIP Kompak Soroti Survei ...,prabowo,"Selasa, 03 Jan 2023 16:47 WIB",Lembaga survei Indo Riset telah merilis hasil ...,detik,https://news.detik.com/pemilu/d-6496532/rahayu...
5,Arahan Prabowo ke Pejabat Kemhan: Pengabdian H...,prabowo,"Selasa, 03 Jan 2023 19:18 WIB",Menteri Pertahanan RI Prabowo Subianto memberi...,detik,https://news.detik.com/berita/d-6496871/arahan...
6,Simulasi Head to Head Capres 2024 Indo Riset: ...,prabowo,"Selasa, 03 Jan 2023 15:13 WIB",Lembaga Indo Riset merilis survei kandidat ter...,detik,https://news.detik.com/pemilu/d-6496241/simula...
7,Kapolri Beri Atensi Khusus Kasus Bocah Diculik...,prabowo,"Selasa, 03 Jan 2023 15:49 WIB",Kadiv Humas Mabes Polri Irjen Dedi Prasetyo me...,detik,https://news.detik.com/berita/d-6496346/kapolr...
8,Legislator Gerindra Serap Aspirasi Driver Ojek...,prabowo,"Rabu, 04 Jan 2023 14:51 WIB",Anggota DPR RI dari Fraksi Partai Gerindra Sit...,detik,https://news.detik.com/berita/d-6498049/legisl...
9,"Elektabilitas Ganjar, Anies dan Prabowo Setahu...",prabowo,"Selasa, 03 Jan 2023 14:37 WIB",Lembaga Indo Riset merilis survei tren elektab...,detik,https://news.detik.com/pemilu/d-6496170/elekta...
10,Gerindra Jatim Pegang Janji Sandiaga Uno untuk...,prabowo,"Rabu, 04 Jan 2023 14:40 WIB",Sandiaga Salahuddin Uno santer diisukan akan p...,detik,https://www.detik.com/jatim/berita/d-6498021/g...


In [ ]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'../tempat_simpan_nasional/detik_{data["keywords"]}_{current_datetime}_baru.xlsx'
filtered_df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ../tempat_simpan_nasional/detik_prabowo_2023-11-11_09-12-30.xlsx
